In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
from requests import Session
import requests

import pandas as pd
import time
import datetime
from tqdm import tqdm
import re
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

# db management libraries
import pymysql
from controller import MysqlController

In [2]:
# 연결
with open('../connection.txt', "r") as f:
                connect_info = list(map(lambda x: x.strip(), f.read().split(",")))
server = MysqlController(*connect_info)

# 주소지 받아오기
ADR = pd.read_sql('SELECT DISTINCT sigungu, dong FROM Address;', server.conn)

In [ ]:
server.conn.close()

# Selenium

In [ ]:
# options = webdriver.ChromeOptions()
# # options.add_argument("--headless")
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

# # Path
# chromedriver_path = '/Users/yejin/Downloads/chromedriver'
# driver = webdriver.Chrome(executable_path = chromedriver_path, options = options)

In [ ]:
# # selenium? 여기서? 굳이?
# url = f'https://www.mangoplate.com/search/{ADR.sigungu.iloc[idx]}%20{ADR.dong.iloc[idx]}?page={p}'
# driver.get(url)

# Bs4

In [3]:
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

In [5]:
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

for idx in range(len(ADR)):
    sigungu = ADR.sigungu.iloc[idx]
    dong = ADR.dong.iloc[idx]
    for page in range(1, 10):
        url = f'https://www.mangoplate.com/search/{sigungu} {dong}?page={page}'
        resp = download(url, headers = headers)
        dom = BeautifulSoup(resp.content, 'lxml')
        fig = dom.select('figure.restaurant-item > figcaption')
        if len(fig) == 0:
            break

        # 식당 한 개 정보
        for i in range(len(fig)):
            res = fig[i]
            res_id = res.select_one('a')['href'].split('/')[2]
            # 식당 한개 사이트 들어감
            one_url = f'https://www.mangoplate.com/restaurants/{res_id}'
            one_resp = download(one_url, headers = headers)
            one_dom = BeautifulSoup(one_resp.content, 'lxml')

            server.insert('mango_restaurants', {
                'updated_at' : datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'mango_id' : res_id, # "/restaurants/~~~" link
                'name': one_dom.select_one('h1.restaurant_name').text,
                'cnt_hit': int(one_dom.select_one('span.cnt.hit').text.replace(',', "")),
                'cnt_review': int(one_dom.select_one('span.cnt.review').text.replace(',', "")),
                'cnt_favorite': int(one_dom.select_one('span.cnt.favorite').text.replace(',', "")),
                'category': one_dom.select_one('table > tbody > tr:nth-of-type(3) > td > span').text, 
                'road_address': one_dom.select_one('div.Restaurant__InfoItemLabel--RoadAddressText').text,
                'address': one_dom.select_one('span.Restaurant__InfoAddress--Text').text,
                'phone': one_dom.select('div.Restaurant__InfoItemLabel')[1].text.strip(),
                'price': one_dom.select_one('table > tbody > tr:nth-of-type(4) > td').text
                })

AttributeError: 'NoneType' object has no attribute 'text'